In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast
import nltk
from nltk.stem.porter import PorterStemmer
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
movies = pd.read_csv("/content/tmdb_5000_movies.csv")
credits = pd.read_csv("/content/tmdb_5000_credits.csv")
movies = movies.merge(credits,on='title')

In [ ]:
movies.head()
movies = movies[['title','genres','keywords','overview','production_companies','release_date','vote_average','vote_count','cast','crew']]

In [ ]:
movies.dropna()
movies.drop_duplicates()

,title,genres,keywords,overview,production_companies,release_date,vote_average,vote_count,cast,crew
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...",2009-12-10,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",2007-05-19,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",2015-10-26,6.3,4466,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",2012-07-16,7.6,9106,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",2012-03-07,6.1,2124,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...,...,...,...
4804,El Mariachi,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...,"[{""name"": ""Columbia Pictures"", ""id"": 5}]",1992-09-04,6.6,238,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,Newlyweds,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],A newlywed couple's honeymoon is upended by th...,[],2011-12-26,5.9,5,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4806,"Signed, Sealed, Delivered","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...","[{""name"": ""Front Street Pictures"", ""id"": 3958}...",2013-10-13,7.0,6,"[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4807,Shanghai Calling,[],[],When ambitious New York attorney Sam is sent t...,[],2012-05-03,5.7,7,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [ ]:
minvotes = movies['vote_count'].quantile(.4)
meanvotes = movies['vote_average'].mean()
movies.drop(movies[movies['vote_count'] < minvotes].index, inplace=True)

In [ ]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+minvotes) * R) + (minvotes/(minvotes+v) * meanvotes)

In [ ]:
movies['rating'] = movies.apply(weighted_rating, axis=1)

In [ ]:
def fix(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'].lower())
    return L

def fixCast(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'].lower())
        if len(L) == 3:
            return L
    return L

def fixCrew(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'].lower())
    return L

def fixDate(date):
    d = ''
    for i in date:
        if i == '-':
            break
        d = d + i
    d = int(d)
    start = d - d%10
    end = start + 10
    d = 'ReYe-'+str(start)+'-'+str(end)
    return [d]

In [ ]:
import string

punct = '!"#$%&\'()|,-./:;<=>?@[\\]^_`{}~'
transtab = str.maketrans(dict.fromkeys(punct, ''))


movies['overview'] = '+*'.join(movies['overview'].tolist()).translate(transtab).split('+*')

In [ ]:
movies['genres'] = movies['genres'].apply(fix)
movies['keywords'] = movies['keywords'].apply(fix)
movies['production_companies'] = movies['production_companies'].apply(fix)
movies['release_date'] = movies['release_date'].apply(fixDate)
movies['cast'] = movies['cast'].apply(fixCast)
movies['crew'] = movies['crew'].apply(fixCrew)
movies['overview'] = movies['overview'].apply(lambda s : s.lower()).apply(lambda s : s.split())
movies.drop(columns=['vote_average','vote_count'], inplace = True)

In [ ]:
movies['new_title'] = movies['title'].apply(lambda s : s.lower()).apply(lambda s : s.split())

In [ ]:
movies

,title,genres,keywords,overview,production_companies,release_date,cast,crew,rating,new_title
0,Avatar,"[action, adventure, fantasy, science fiction]","[culture clash, future, space war, space colon...","[in, the, 22nd, century, a, paraplegic, marine...","[ingenious film partners, twentieth century fo...",[ReYe-2000-2010],"[sam worthington, zoe saldana, sigourney weaver]",[james cameron],7.187106,[avatar]
1,Pirates of the Caribbean: At World's End,"[adventure, fantasy, action]","[ocean, drug abuse, exotic island, east india ...","[captain, barbossa, long, believed, to, be, de...","[walt disney pictures, jerry bruckheimer films...",[ReYe-2000-2010],"[johnny depp, orlando bloom, keira knightley]",[gore verbinski],6.875805,"[pirates, of, the, caribbean:, at, world's, end]"
2,Spectre,"[action, adventure, crime]","[spy, based on novel, secret agent, sequel, mi...","[a, cryptic, message, from, bond’s, past, send...","[columbia pictures, danjaq, b24]",[ReYe-2010-2020],"[daniel craig, christoph waltz, léa seydoux]",[sam mendes],6.293737,[spectre]
3,The Dark Knight Rises,"[action, crime, drama, thriller]","[dc comics, crime fighter, terrorist, secret i...","[following, the, death, of, district, attorney...","[legendary pictures, warner bros., dc entertai...",[ReYe-2010-2020],"[christian bale, michael caine, gary oldman]",[christopher nolan],7.577335,"[the, dark, knight, rises]"
4,John Carter,"[action, adventure, science fiction]","[based on novel, mars, medallion, space travel...","[john, carter, is, a, warweary, former, milita...",[walt disney pictures],[ReYe-2010-2020],"[taylor kitsch, lynn collins, samantha morton]",[andrew stanton],6.099540,"[john, carter]"
...,...,...,...,...,...,...,...,...,...,...
4748,Super Size Me,"[documentary, comedy, drama]","[experiment, health, junk food, food industry,...","[morgan, spurlock, subjects, himself, to, a, d...",[kathbur pictures],[ReYe-2000-2010],"[morgan spurlock, daryl isaacs, lisa ganjhu]",[morgan spurlock],6.490635,"[super, size, me]"
4764,The Signal,"[thriller, science fiction]","[hacker, supernatural powers, road trip, indep...","[three, college, students, on, a, road, trip, ...","[automatik entertainment, low spark films, sig...",[ReYe-2010-2020],"[brenton thwaites, olivia cooke, beau knapp]",[william eubank],5.852804,"[the, signal]"
4779,Clerks,[comedy],"[salesclerk, loser, aftercreditsstinger]","[convenience, and, video, store, clerks, dante...","[miramax films, view askew productions]",[ReYe-1990-2000],"[brian o'halloran, jeff anderson, jason mewes]",[kevin smith],7.196711,[clerks]
4802,Primer,"[science fiction, drama, thriller]","[distrust, garage, identity crisis, time trave...","[friendsfledgling, entrepreneurs, invent, a, d...",[thinkfilm],[ReYe-2000-2010],"[shane carruth, david sullivan, casey gooden]",[shane carruth],6.759171,[primer]


In [ ]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)
movies['production_companies'] = movies['production_companies'].apply(collapse)

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
def stemmer(text):
    y = []
    for i in text:
        y.append(ps.stem(i))
    return y

In [ ]:
movies['overview'] = movies['overview'].apply(stemmer)

In [ ]:
movies['tags'] = movies['genres'] + movies['keywords'] + movies['overview'] + movies['production_companies'] + movies['release_date'] + movies['cast'] + movies['crew'] + movies['crew'] + movies['crew'] + movies['new_title']
movies['tags'] = movies['tags'].apply(lambda x : ' '.join(x))

In [ ]:
movies.drop(columns = ['genres', 'keywords', 'overview', 'production_companies', 'release_date', 'cast', 'crew', 'new_title'], inplace =True)

In [ ]:
movies = movies.reset_index(drop=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [ ]:
tf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tf.fit_transform(movies['tags'])

In [ ]:
sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
sim

array([[1.00000000e+00, 1.24420727e-02, 6.81295369e-03, ...,
        1.69816029e-03, 1.98909448e-02, 3.03852843e-03],
       [1.24420727e-02, 1.00000000e+00, 7.17253835e-03, ...,
        9.90732149e-03, 5.39316188e-03, 6.19529542e-03],
       [6.81295369e-03, 7.17253835e-03, 1.00000000e+00, ...,
        5.34680433e-04, 1.14827990e-03, 1.10791709e-02],
       ...,
       [1.69816029e-03, 9.90732149e-03, 5.34680433e-04, ...,
        1.00000000e+00, 1.20151253e-02, 1.25968993e-02],
       [1.98909448e-02, 5.39316188e-03, 1.14827990e-03, ...,
        1.20151253e-02, 1.00000000e+00, 1.05926170e-02],
       [3.03852843e-03, 6.19529542e-03, 1.10791709e-02, ...,
        1.25968993e-02, 1.05926170e-02, 1.00000000e+00]])

In [ ]:
def recommend(movie):
    index = movies[movies['title'] == movie].index[0]
    distances = sorted(list(enumerate(sim[index])),reverse=True,key = lambda x: x[1])
    answer = []
    for i in distances[1:6]:
        answer.append((movies.iloc[i[0]].title, movies.iloc[i[0]].rating))
    answer = sorted(answer,reverse=True,key = lambda x: x[1])
    for i in answer:
        print(i[0])

In [ ]:
recommend("Avatar")

Terminator 2: Judgment Day
Aliens
The Terminator
The Abyss
True Lies
